In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

print("IMPORT")
import tensorflow as tf #tensorflow
print(tf.__version__)
import numpy as np #numpy > save loss .. 
from collections import OrderedDict #layer ..
import os, random #dir, random..
import pickle #save & load

# cifar 10 
#wget http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
#tar xzf cifar-10-python.tar.gz

#resnet
#https://arxiv.org/abs/1512.03385

IMPORT
1.0.1


In [11]:
print("VARIABLES")
#path
DATA_PATH = 'cifar10/cifar-10-batches-py/'
CHECKPOINT_PATH = 'checkpoint/simple_res_net/simple_res_net.ckpt'
SAVE_PATH = 'checkpoint/simple_res_net/'

#parameters
INPUT_SIDE = 32
INPUT_SIZE = INPUT_SIDE * INPUT_SIDE
N_CHANNEL = 3
BATCH_SIZE = 64
EPOCHS = 1000
LR = 0.01
LR_DECAY_RATE = 0.5

LIST_CLASS=['airplane', 'automobile', 'birds', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
N_CLASSES = len(LIST_CLASS)

#train&test batch
train_file=['data_batch_1','data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']
test_file=['cifar10/cifar-10-batches-py/test_batch']

#input & output 
x = tf.placeholder("float", [BATCH_SIZE, INPUT_SIZE * N_CHANNEL]) #batch x image size
y = tf.placeholder("float", [BATCH_SIZE, N_CLASSES]) #batch x class`


print("input image size : {}".format(INPUT_SIZE))
print("image channel : {}".format(N_CHANNEL))
print("batch size : {}".format(BATCH_SIZE))
print("num of class : {}".format(N_CLASSES))
print("training epochs : {}".format(EPOCHS))
print("learning rate : {}".format(LR))
print("learning decay rate : {}".format(LR_DECAY_RATE))

VARIABLES
input image size : 1024
image channel : 3
batch size : 64
num of class : 10
training epochs : 1000
learning rate : 0.01
learning decay rate : 0.5


In [12]:
#NETWORK PARAMETERS

print("NETWORK PARAMETERS")

stddev = 0.1

weights = {
     'conv' : tf.Variable(tf.random_normal([3, 3, N_CHANNEL, 16], stddev=stddev)),
    'conv1_1x1' : tf.Variable(tf.random_normal([1, 1, 16, 4], stddev=stddev)),
    'conv1_3x3' : tf.Variable(tf.random_normal([3, 3, 4, 4], stddev=stddev)),
    'conv1_1x1_16' : tf.Variable(tf.random_normal([1, 1, 4, 16], stddev=stddev)),
    'conv2_1x1' : tf.Variable(tf.random_normal([1, 1, 16, 4], stddev=stddev)),
    'conv2_3x3' : tf.Variable(tf.random_normal([3, 3, 4, 4], stddev=stddev)),
    'conv2_1x1_16' : tf.Variable(tf.random_normal([1, 1, 4, 16], stddev=stddev)),
    
    # conv 16 + conv2 16 = 32filters 
    
    'conv3_1x1' : tf.Variable(tf.random_normal([1, 1, 32, 8], stddev=stddev)),
    'conv3_3x3' : tf.Variable(tf.random_normal([3, 3, 8, 8], stddev=stddev)),
    'conv3_1x1_32' : tf.Variable(tf.random_normal([1, 1, 8, 32], stddev=stddev)),
    'conv4_1x1' : tf.Variable(tf.random_normal([1, 1, 32, 8], stddev=stddev)),
    'conv4_3x3' : tf.Variable(tf.random_normal([3, 3, 8, 8], stddev=stddev)),
    'conv4_1x1_32' : tf.Variable(tf.random_normal([1, 1, 8, 32], stddev=stddev)),
    
    # conv2 지난거32 + conv4 32 = 64
    
    'conv5_1x1' : tf.Variable(tf.random_normal([1, 1, 64, 16], stddev=stddev)),
    'conv5_3x3' : tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev)),
    'conv5_1x1_64' : tf.Variable(tf.random_normal([1, 1, 16, 64], stddev=stddev)),
    'conv6_1x1' : tf.Variable(tf.random_normal([1, 1, 64, 16], stddev=stddev)),
    'conv6_3x3' : tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev)),
    'conv6_1x1_64' : tf.Variable(tf.random_normal([1, 1, 16, 64], stddev=stddev)),
    
    #conv4 지난거 64 + conv6 64 = 128
    
    'dense1' : tf.Variable(tf.random_normal([16*16*128, 1000], stddev=stddev)),
    'dense2' : tf.Variable(tf.random_normal([1000, N_CLASSES], stddev=stddev))
    
    
}
biases = {
    'conv' : tf.Variable(tf.random_normal([16], stddev=stddev)),
    'conv1_1x1' : tf.Variable(tf.random_normal([4], stddev=stddev)),
    'conv1_3x3' : tf.Variable(tf.random_normal([4], stddev=stddev)),
    'conv1_1x1_16' : tf.Variable(tf.random_normal([16], stddev=stddev)),
    'conv2_1x1' : tf.Variable(tf.random_normal([4], stddev=stddev)),
    'conv2_3x3' :tf.Variable(tf.random_normal([4], stddev=stddev)),
    'conv2_1x1_16' : tf.Variable(tf.random_normal([16], stddev=stddev)),
    
    'conv3_1x1' : tf.Variable(tf.random_normal([8], stddev=stddev)),
    'conv3_3x3' : tf.Variable(tf.random_normal([8], stddev=stddev)),
    'conv3_1x1_32' : tf.Variable(tf.random_normal([32], stddev=stddev)),
    'conv4_1x1' : tf.Variable(tf.random_normal([8], stddev=stddev)),
    'conv4_3x3' : tf.Variable(tf.random_normal([8], stddev=stddev)),
    'conv4_1x1_32' : tf.Variable(tf.random_normal([32], stddev=stddev)),
    
    'conv5_1x1' : tf.Variable(tf.random_normal([16], stddev=stddev)),
    'conv5_3x3' : tf.Variable(tf.random_normal([16], stddev=stddev)),
    'conv5_1x1_64' : tf.Variable(tf.random_normal([64], stddev=stddev)),
    'conv6_1x1' : tf.Variable(tf.random_normal([16], stddev=stddev)),
    'conv6_3x3' : tf.Variable(tf.random_normal([16], stddev=stddev)),
    'conv6_1x1_64' : tf.Variable(tf.random_normal([64], stddev=stddev)),
    
    'dense1' : tf.Variable(tf.random_normal([1000], stddev=stddev)),
    'dense2' : tf.Variable(tf.random_normal([N_CLASSES], stddev=stddev))
    
}

w = {
   
    'conv' : tf.Variable(tf.random_normal([3, 3, N_CHANNEL, 8], stddev=stddev)),
    'conv1_1x1' : tf.Variable(tf.random_normal([1, 1, 8, 2], stddev=stddev)),
    'conv1_3x3' : tf.Variable(tf.random_normal([3, 3, 2, 2], stddev=stddev)),
    'conv1_1x1_16' : tf.Variable(tf.random_normal([1, 1, 2, 8], stddev=stddev)),
    'conv2_1x1' : tf.Variable(tf.random_normal([1, 1, 8, 2], stddev=stddev)),
    'conv2_3x3' : tf.Variable(tf.random_normal([3, 3, 2, 2], stddev=stddev)),
    'conv2_1x1_16' : tf.Variable(tf.random_normal([1, 1, 2, 8], stddev=stddev)),
    
    # conv 8 + conv2 8= 16filters 
    
    'conv3_1x1' : tf.Variable(tf.random_normal([1, 1, 16, 4], stddev=stddev)),
    'conv3_3x3' : tf.Variable(tf.random_normal([3, 3, 4, 4], stddev=stddev)),
    'conv3_1x1_32' : tf.Variable(tf.random_normal([1, 1, 4, 16], stddev=stddev)),
    'conv4_1x1' : tf.Variable(tf.random_normal([1, 1, 16, 4], stddev=stddev)),
    'conv4_3x3' : tf.Variable(tf.random_normal([3, 3, 4, 4], stddev=stddev)),
    'conv4_1x1_32' : tf.Variable(tf.random_normal([1, 1, 4, 16], stddev=stddev)),
    
    '''
    # conv2 지난거16 + conv4 16 = 32
    
    'conv5_1x1' : tf.Variable(tf.random_normal([1, 1, 32, 8], stddev=stddev)),
    'conv5_3x3' : tf.Variable(tf.random_normal([3, 3, 8, 8], stddev=stddev)),
    'conv5_1x1_64' : tf.Variable(tf.random_normal([1, 1, 8, 32], stddev=stddev)),
    'conv6_1x1' : tf.Variable(tf.random_normal([1, 1, 32, 8], stddev=stddev)),
    'conv6_3x3' : tf.Variable(tf.random_normal([3, 3, 8, 8], stddev=stddev)),
    'conv6_1x1_64' : tf.Variable(tf.random_normal([1, 1, 8, 32], stddev=stddev)),
    
    #conv4 지난거 32 + conv6 32 = 64
    '''
    
    
    'dense1' : tf.Variable(tf.random_normal([16*16*32, 1000], stddev=stddev)),
    'dense2' : tf.Variable(tf.random_normal([1000, N_CLASSES], stddev=stddev))
}
b = { 
  
    'conv' : tf.Variable(tf.random_normal([8], stddev=stddev)),
    'conv1_1x1' : tf.Variable(tf.random_normal([2], stddev=stddev)),
    'conv1_3x3' : tf.Variable(tf.random_normal([2], stddev=stddev)),
    'conv1_1x1_16' : tf.Variable(tf.random_normal([8], stddev=stddev)),
    'conv2_1x1' : tf.Variable(tf.random_normal([2], stddev=stddev)),
    'conv2_3x3' :tf.Variable(tf.random_normal([2], stddev=stddev)),
    'conv2_1x1_16' : tf.Variable(tf.random_normal([8], stddev=stddev)),
    
    'conv3_1x1' : tf.Variable(tf.random_normal([4], stddev=stddev)),
    'conv3_3x3' : tf.Variable(tf.random_normal([4], stddev=stddev)),
    'conv3_1x1_32' : tf.Variable(tf.random_normal([16], stddev=stddev)),
    'conv4_1x1' : tf.Variable(tf.random_normal([4], stddev=stddev)),
    'conv4_3x3' : tf.Variable(tf.random_normal([4], stddev=stddev)),
    'conv4_1x1_32' : tf.Variable(tf.random_normal([16], stddev=stddev)),
    
    'conv5_1x1' : tf.Variable(tf.random_normal([8], stddev=stddev)),
    'conv5_3x3' : tf.Variable(tf.random_normal([8], stddev=stddev)),
    'conv5_1x1_64' : tf.Variable(tf.random_normal([32], stddev=stddev)),
    'conv6_1x1' : tf.Variable(tf.random_normal([8], stddev=stddev)),
    'conv6_3x3' : tf.Variable(tf.random_normal([8], stddev=stddev)),
    'conv6_1x1_64' : tf.Variable(tf.random_normal([32], stddev=stddev)),
    
    'dense1' : tf.Variable(tf.random_normal([1000], stddev=stddev)),
    'dense2' : tf.Variable(tf.random_normal([N_CLASSES], stddev=stddev))
}

NETWORK PARAMETERS


In [13]:
#save
saver = tf.train.Saver(max_to_keep=3)
save_step = 100 #save for 100 epoch
if not os.path.exists(CHECKPOINT_PATH):
    os.makedirs(CHECKPOINT_PATH)
print ("SAVER READY")

SAVER READY


In [14]:
#model
#http://laonple.blog.me/220764986252 - bottleneck 
#conv 3x3
#conv (1x1, 3x3, 1x1) - relu > 16
#conv (1x1, 3x3, 1x1) relu > 32
#conv (1x1, 3x3, 1x1) relu > 64
#avg pooling 
#fc
#softmax

print("NETWORK FUNCTION")

def SimpleResNet(img_width, img_height, img_channel, _x, _w, _b, scope='SimpleResNet'):
    network = OrderedDict() #network layers

    # X RESHAPE
    _x_r = tf.reshape(_x, shape=[-1,img_width,img_height, img_channel])
    
    with tf.variable_scope(scope):
        conv = tf.nn.conv2d(_x_r, _w['conv'], strides=[1, 1, 1, 1], padding='SAME')
        conv = tf.nn.bias_add(conv, _b['conv'])
        conv = tf.nn.relu(conv)
        network['conv'] = conv
        
        #16
        conv1_1x1 = tf.nn.conv2d(conv, _w['conv1_1x1'], strides=[1, 1, 1, 1], padding='SAME')
        conv1_1x1 = tf.nn.bias_add(conv1_1x1, _b['conv1_1x1'])
        conv1_1x1 = tf.nn.relu(conv1_1x1)
        conv1_3x3 = tf.nn.conv2d(conv1_1x1, _w['conv1_3x3'], strides=[1, 1, 1, 1], padding='SAME')
        conv1_3x3 = tf.nn.bias_add(conv1_3x3, _b['conv1_3x3'])
        conv1_3x3 = tf.nn.relu(conv1_3x3)
        conv1_1x1_16 = tf.nn.conv2d(conv1_3x3, _w['conv1_1x1_16'], strides=[1, 1, 1, 1], padding='SAME')
        conv1_1x1_16 = tf.nn.bias_add(conv1_1x1_16, _b['conv1_1x1_16'])
        conv1_1x1_16 = tf.nn.relu(conv1_1x1_16)
        network['conv1_1x1_16'] = conv1_1x1_16
        
        #16
        conv2_1x1 = tf.nn.conv2d(conv1_1x1_16, _w['conv2_1x1'], strides=[1, 1, 1, 1], padding='SAME')
        conv2_1x1 = tf.nn.bias_add(conv2_1x1, _b['conv2_1x1'])
        conv2_1x1 = tf.nn.relu(conv2_1x1)
        conv2_3x3 = tf.nn.conv2d(conv2_1x1, _w['conv2_3x3'], strides=[1, 1, 1, 1], padding='SAME')
        conv2_3x3 = tf.nn.bias_add(conv2_3x3, _b['conv2_3x3'])
        conv2_3x3 = tf.nn.relu(conv2_3x3)
        conv2_1x1_16 = tf.nn.conv2d(conv2_3x3, _w['conv2_1x1_16'], strides=[1, 1, 1, 1], padding='SAME')
        conv2_1x1_16 = tf.nn.bias_add(conv2_1x1_16, _b['conv2_1x1_16'])
        #print(conv2_1x1_16.shape)
        #32
        conv2_1x1_16 = tf.concat([conv, conv2_1x1_16], 3) 
        conv2_1x1_16 = tf.nn.relu(conv2_1x1_16)
        network['conv2_1x1_16'] = conv2_1x1_16
       
        #print(conv2_1x1_16.shape)
        #32       32 x 32 x 16+16 >>> 
        conv3_1x1 = tf.nn.conv2d(conv2_1x1_16, _w['conv3_1x1'], strides=[1, 1, 1, 1], padding='SAME')
        conv3_1x1 = tf.nn.bias_add(conv3_1x1, _b['conv3_1x1'])
        conv3_1x1 = tf.nn.relu(conv3_1x1)
        conv3_3x3 = tf.nn.conv2d(conv3_1x1, _w['conv3_3x3'], strides=[1, 1, 1, 1], padding='SAME')
        conv3_3x3 = tf.nn.bias_add(conv3_3x3, _b['conv3_3x3'])
        conv3_3x3 = tf.nn.relu(conv3_3x3)
        conv3_1x1_32 = tf.nn.conv2d(conv3_3x3, _w['conv3_1x1_32'], strides=[1, 1, 1, 1], padding='SAME')
        conv3_1x1_32 = tf.nn.bias_add(conv3_1x1_32, _b['conv3_1x1_32'])
        conv3_1x1_32 = tf.nn.relu(conv3_1x1_32)
        network['conv3_1x1_32'] = conv3_1x1_32
        
        conv4_1x1 = tf.nn.conv2d(conv3_1x1_32, _w['conv4_1x1'], strides=[1, 1, 1, 1], padding='SAME')
        conv4_1x1 = tf.nn.bias_add(conv4_1x1, _b['conv4_1x1'])
        conv4_1x1 = tf.nn.relu(conv4_1x1)
        conv4_3x3 = tf.nn.conv2d(conv4_1x1, _w['conv4_3x3'], strides=[1, 1, 1, 1], padding='SAME')
        conv4_3x3 = tf.nn.bias_add(conv4_3x3, _b['conv4_3x3'])
        conv4_3x3 = tf.nn.relu(conv4_3x3)
        conv4_1x1_32 = tf.nn.conv2d(conv4_3x3, _w['conv4_1x1_32'], strides=[1, 1, 1, 1], padding='SAME')
        conv4_1x1_32 = tf.nn.bias_add(conv4_1x1_32, _b['conv4_1x1_32'])
        
        #64
        conv4_1x1_32 = tf.concat([conv2_1x1_16, conv4_1x1_32 ], 3)
        conv4_1x1_32 = tf.nn.relu(conv4_1x1_32)
        network['conv4_1x1_32'] = conv4_1x1_32
    
        conv5_1x1 = tf.nn.conv2d(conv4_1x1_32, _w['conv5_1x1'], strides=[1, 1, 1, 1], padding='SAME')
        conv5_1x1 = tf.nn.bias_add(conv5_1x1, _b['conv5_1x1'])
        conv5_1x1 = tf.nn.relu(conv5_1x1)
        conv5_3x3 = tf.nn.conv2d(conv5_1x1, _w['conv5_3x3'], strides=[1, 1, 1, 1], padding='SAME')
        conv5_3x3 = tf.nn.bias_add(conv5_3x3, _b['conv5_3x3'])
        conv5_3x3 = tf.nn.relu(conv5_3x3)
        conv5_1x1_64 = tf.nn.conv2d(conv5_3x3, _w['conv5_1x1_64'], strides=[1, 1, 1, 1], padding='SAME')
        conv5_1x1_64 = tf.nn.bias_add(conv5_1x1_64, _b['conv5_1x1_64'])
        conv5_1x1_64 = tf.nn.relu(conv5_1x1_64)
        network['conv5_1x1_64'] = conv5_1x1_64
        
        conv6_1x1 = tf.nn.conv2d(conv5_1x1_64, _w['conv6_1x1'], strides=[1, 1, 1, 1], padding='SAME')
        conv6_1x1 = tf.nn.bias_add(conv6_1x1, _b['conv6_1x1'])
        conv6_1x1 = tf.nn.relu(conv6_1x1)
        conv6_3x3 = tf.nn.conv2d(conv6_1x1, _w['conv6_3x3'], strides=[1, 1, 1, 1], padding='SAME')
        conv6_3x3 = tf.nn.bias_add(conv6_3x3, _b['conv6_3x3'])
        conv6_3x3 = tf.nn.relu(conv6_3x3)
        conv6_1x1_64 = tf.nn.conv2d(conv6_3x3, _w['conv6_1x1_64'], strides=[1, 1, 1, 1], padding='SAME')
        conv6_1x1_64 = tf.nn.bias_add(conv6_1x1_64, _b['conv6_1x1_64'])
        
        #128
        conv6_1x1_64 = tf.concat([conv4_1x1_32, conv6_1x1_64],3)
        conv6_1x1_64 = tf.nn.relu(conv6_1x1_64)
        network['conv6_1x1_64'] = conv6_1x1_64
        
        pool = tf.nn.avg_pool(conv4_1x1_32, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        #pool = tf.nn.avg_pool(conv6_1x1_64, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        network['pool'] = pool
        
        dense = tf.reshape(pool, [-1, _w['dense1'].get_shape().as_list()[0]])
        dense1 = tf.add(tf.matmul(dense, _w['dense1']), _b['dense1'])
        dense1 = tf.nn.relu(dense1)
        network['dense1'] = dense1
        
        logit = tf.add(tf.matmul(dense1, _w['dense2']), _b['dense2'])
        network['logit'] = logit
        
    return network

NETWORK FUNCTION


In [15]:
out = SimpleResNet(INPUT_SIDE, INPUT_SIDE, N_CHANNEL, x, weights, biases, 'SimpleResNet')
print("CHECK LAYERS")
for key, value in out.items():
    print (key, value)

CHECK LAYERS
conv Tensor("SimpleResNet_2/Relu:0", shape=(64, 32, 32, 16), dtype=float32)
conv1_1x1_16 Tensor("SimpleResNet_2/Relu_3:0", shape=(64, 32, 32, 16), dtype=float32)
conv2_1x1_16 Tensor("SimpleResNet_2/Relu_6:0", shape=(64, 32, 32, 32), dtype=float32)
conv3_1x1_32 Tensor("SimpleResNet_2/Relu_9:0", shape=(64, 32, 32, 32), dtype=float32)
conv4_1x1_32 Tensor("SimpleResNet_2/Relu_12:0", shape=(64, 32, 32, 64), dtype=float32)
conv5_1x1_64 Tensor("SimpleResNet_2/Relu_15:0", shape=(64, 32, 32, 64), dtype=float32)
conv6_1x1_64 Tensor("SimpleResNet_2/Relu_18:0", shape=(64, 32, 32, 128), dtype=float32)
pool Tensor("SimpleResNet_2/AvgPool:0", shape=(64, 16, 16, 64), dtype=float32)
dense1 Tensor("SimpleResNet_2/Relu_19:0", shape=(32, 1000), dtype=float32)
logit Tensor("SimpleResNet_2/Add_1:0", shape=(32, 10), dtype=float32)


In [17]:
#cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out['logit']))
tf.summary.scalar("loss", cost)
print("LOSS FUNCTION")

#learning rate
global_step = tf.Variable(0, trainable=False, name="global_step")
learning_rate = tf.train.exponential_decay(learning_rate=LR,
                                           global_step=global_step,
                                           decay_steps=50000,
                                           decay_rate=LR_DECAY_RATE,
                                           staircase=True,
                                           name="learning_rate")
learning_rate = tf.maximum(learning_rate, 0.0001)
tf.summary.scalar("learning_rate", learning_rate)
print("LERANING RATE : {}".format(learning_rate))

#optimizer
adam = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
sgd = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)
print("OPTIMIZER 1 : {}".format(adam))
print("OPTIMIZER 2 : {}".format(sgd))

corr = tf.equal(tf.argmax(out['logit'], 1), tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(corr, "float"))

ValueError: Dimension 0 in both shapes must be equal, but are 32 and 64 for 'SoftmaxCrossEntropyWithLogits_1' (op: 'SoftmaxCrossEntropyWithLogits') with input shapes: [32,10], [64,10].

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)


#summary_writer = tf.summary.FileWriter(CHECKPOINT_PATH, sess.graph)
#checkpoint = tf.train.latesrt_checkpoint(CHECKPOINT_PATH)

#포문돌다 세이브할땐
#save_path = saver.save(sess, CHECKPOINT_PATH + "mm.ckpt", global_step=global_step)

In [ ]:
#training
for epoch in range(EPOCHS):
    

In [ ]:
predictor = prediction.Predictor(checkpoint_file)
predictions, scores, bboxes = predictor.predict(os.path.join(image_dir, image_file_name), sliding_windows)
predictions = np.array(predictions)

In [ ]:
queue_positives = tf.RandomShuffleQueue(10000, 
                                            min_after_dequeue=5000,
                                            dtypes=[tf.float32, tf.int32], 
                                            shapes=[[PATCH_HEIGHT, PATCH_WIDTH, 1], []])
    
    queue_negatives = tf.RandomShuffleQueue(10000, 
                                            min_after_dequeue=5000,
                                            dtypes=[tf.float32, tf.int32], 
                                            shapes=[[PATCH_HEIGHT, PATCH_WIDTH, 1], []])
    
    filename_placeholder = tf.placeholder(tf.string, name="filename")    
    labels_placeholder = tf.placeholder(tf.int32, shape=[None, ], name="labels")
    bboxes_placeholder = tf.placeholder(tf.int32, shape=[None, 4], name="bboxes_placeholder")
    
    encoded = tf.read_file(filename_placeholder)
    image = tf.image.decode_image(encoded)

In [ ]:
def prepare_patch(bbox):        
        patch = tf.image.crop_to_bounding_box(image, bbox[1], bbox[0], bbox[3], bbox[2])
        patch = tf.image.per_image_standardization(patch)
        patch = tf.image.resize_images(patch, [PATCH_HEIGHT, PATCH_WIDTH])
        patch = tf.image.random_flip_left_right(patch)
        patch = tf.image.random_flip_up_down(patch)    
        patch = tf.image.random_brightness(patch, 0.15)
        patch.set_shape([PATCH_HEIGHT, PATCH_WIDTH, 1])
        return patch

In [ ]:
patches = tf.map_fn(prepare_patch, bboxes_placeholder, dtype=tf.float32)
    negative_patches, positive_patches = tf.dynamic_partition(patches, labels_placeholder, 2)
    negative_labels, positive_labels = tf.dynamic_partition(labels_placeholder, labels_placeholder, 2)
    
    enqueue_negatives = queue_negatives.enqueue_many([negative_patches, negative_labels])
    enqueue_positives = queue_positives.enqueue_many([positive_patches, positive_labels])
    enqueue_op = tf.group(enqueue_negatives, enqueue_positives)

In [ ]:
negative_images, negative_labels = tf.train.batch([negative_image, negative_label], 
                                                      batch_size=128,
                                                      num_threads=4,
                                                      capacity=100,
                                                      shapes=[[PATCH_HEIGHT, PATCH_WIDTH, 1], []])

    positive_images, positive_labels = tf.train.batch([positive_image, positive_label], 
                                                      batch_size=128,
                                                      num_threads=4,
                                                      capacity=100,
                                                      shapes=[[PATCH_HEIGHT, PATCH_WIDTH, 1], []])

In [ ]:
logits = network.network(negative_images, 2)
losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=negative_labels, logits=logits)
_, indices = tf.nn.top_k(losses, REGION_PER_IMAGE // 4, sorted=False)
negative_images = tf.gather(negative_images, indices)
negative_labels = tf.gather(negative_labels, indices)

logits = network.network(positive_images, 2, reuse=True)
losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=positive_labels, logits=logits)
_, indices = tf.nn.top_k(losses, REGION_PER_IMAGE // 4, sorted=False)
positive_images = tf.gather(positive_images, indices)
positive_labels = tf.gather(positive_labels, indices)

images = tf.concat([negative_images, positive_images], axis=0)
labels = tf.concat([negative_labels, positive_labels], axis=0)

# tf.summary.image("filtered_images", images, max_outputs=REGION_PER_IMAGE // 2)

logits = network.network(images, 2, is_training=True, reuse=True)
tf.losses.sparse_softmax_cross_entropy(labels, logits)
total_loss = tf.losses.get_total_loss()
tf.summary.scalar("total_loss", total_loss)

global_step = tf.Variable(0, trainable=False, name="global_step")
learning_rate = tf.train.exponential_decay(learning_rate=LR, 
                                           global_step=global_step, 
                                           decay_steps=50000,
                                           decay_rate=LR_DECAY_RATE, 
                                           staircase=True,
                                           name="learning_rate")
learning_rate = tf.maximum(learning_rate, 0.0001)
tf.summary.scalar("learning_rate", learning_rate)


In [ ]:
optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9, use_nesterov=True)
train_op = tf.contrib.training.create_train_op(total_loss, optimizer, global_step=global_step)

In [ ]:

def prepare_evaluations(filename):
    encoded = tf.read_file(filename)
    image = tf.image.decode_image(encoded)
    image = tf.image.per_image_standardization(image)
    image.set_shape([PATCH_HEIGHT, PATCH_WIDTH, 1])
    return image

In [ ]:
        feed_dict = {filename_placeholder: "{}/{}".format(TRAIN_DIR, filename),
                     labels_placeholder: labels,
                     bboxes_placeholder: bounding_boxes}
        

In [ ]:
        sess.run(enqueue_op, feed_dict=feed_dict)        
                

In [ ]:

def network(inputs, num_outputs, weight_decay=0.0005, is_training=False, reuse=False, scope="network"):    
    with tf.variable_scope(scope, values=[inputs], reuse=reuse):                  
        # layer1
        net = tf.layers.conv2d(inputs, 64, (5, 5), 
                               use_bias=False,                               
                               kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), 
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(weight_decay), 
                               name="conv1")
        net = tf.layers.batch_normalization(net,                                                         
                                            training=is_training, 
                                            name="batch_norm1")
        net = tf.nn.relu(net)
        net = tf.layers.max_pooling2d(net, (2, 2), 2, name="pool1")
        
        # layer2
        net = tf.layers.conv2d(net, 128, (3, 3), 
                               use_bias=False,                                                                     
                               kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), 
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(weight_decay), 
                               name="conv2")
        net = tf.layers.batch_normalization(net,             
                                            training=is_training, 
                                            name="batch_norm2")
        net = tf.nn.relu(net)
        net = tf.layers.max_pooling2d(net, (2, 2), 2, name="pool2")
        
        # layer3
        net = tf.layers.conv2d(net, 192, (3, 3), 
                               use_bias=False,        
                               kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), 
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(weight_decay), 
                               name="conv3")
        net = tf.layers.batch_normalization(net,      
                                            training=is_training, 
                                            name="batch_norm3")
        net = tf.nn.relu(net)
        net = tf.layers.max_pooling2d(net, (2, 2), 2, name="pool3")

In [ ]:
 
        # layer5
        net = tf.layers.conv2d(net, 192, (3, 3), 
                               padding="same",
                               use_bias=False,        
                               kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(), 
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(weight_decay),
                               name="conv5")
        net = tf.layers.batch_normalization(net,      
                                            training=is_training, 
                                            name="batch_norm5")
        net = tf.nn.relu(net)
        net = tf.layers.max_pooling2d(net, (2, 2), 2, name="pool5")
        net = tf.contrib.layers.flatten(net, scope="pool5_flatten")
        
        # layer6
        net = tf.layers.dense(net, 3072, 
                              use_bias=False, 
                              kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                              kernel_regularizer=tf.contrib.layers.l2_regularizer(weight_decay),
                              name="fc6")        
        net = tf.layers.batch_normalization(net, 
                                            training=is_training, 
                                            name="batch_norm6")        
        net = tf.nn.relu(net)
        
        # layer7
        net = tf.layers.dense(net, 1024, 
                              use_bias=False, 
                              kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                              kernel_regularizer=tf.contrib.layers.l2_regularizer(weight_decay),
                              name="fc7")        
        net = tf.nn.relu(net)        
        if is_training:
            net = tf.nn.dropout(net, 0.5, name="dropout7")

        # layer8
        return tf.layers.dense(net, num_outputs, 
                               use_bias=True, 
                               kernel_initializer=tf.contrib.layers.xavier_initializer(), 
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(weight_decay),
                               bias_initializer=tf.zeros_initializer(),
                               name="fc8")

In [ ]:
IMAGE_WIDTH = 6576
IMAGE_HEIGHT = 4384
PATCH_WIDTH = 128
PATCH_HEIGHT = 128
REGION_PER_IMAGE = 128
LR = 0.01
LR_DECAY_RATE = 0.5


In [ ]:
 logits = networks(images, 2, is_training=True, reuse=True)
        tf.losses.sparse_softmax_cross_entropy(labels, logits)
        total_loss = tf.losses.get_total_loss()
        tf.summary.scalar("total_loss", total_loss)

        global_step = tf.Variable(0, trainable=False, name="global_step")
        learning_rate = tf.train.exponential_decay(learning_rate=LR,
                                                   global_step=global_step,
                                                   decay_steps=50000,
                                                   decay_rate=LR_DECAY_RATE,
                                                   staircase=True,
                                                   name="learning_rate")
        learning_rate = tf.maximum(learning_rate, 0.0001)
        tf.summary.scalar("learning_rate", learning_rate)

        optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9, use_nesterov=True)
        train_op = tf.contrib.training.create_train_op(total_loss, optimizer, global_step=global_step)

        def prepare_evaluations(filename):
            encoded = tf.read_file(filename)
            image = tf.image.decode_image(encoded)
            image = tf.image.per_image_standardization(image)
            image.set_shape([PATCH_HEIGHT, PATCH_WIDTH, 1])
            return image

        filenames_placeholder = tf.placeholder(tf.string, shape=[None, ], name="eval_filenames")
        images = tf.map_fn(prepare_evaluations, filenames_placeholder, dtype=tf.float32)


In [1]:
어그맨테이션
오버피팅 제거기술
파인튜닝 마저 하기 
텐서보드
  miss classfication 된 아이디를 저장해둬볼것
        주로 뭐가 가장 많이 미쓰나는지
        분류를 잘못했는지 데이터가 부족한지 결론낼수잇으.

        비쥬얼라이제이션
        피쳐 중간에 나오는거 어떻게 나오는지 이런거 확인
in queue 이거 해보기 



checkpoint
        학습기법
        sgd

        pool 제거
        decay

        인셉션 모듈 추가
        채널 +

        어그멘테이션

        세이브

        오버피팅 제외하는거 드랍아웃이나 배치놈등등
        배치 작게
        64개
        32개정도 960

        dense레이어 갯수

        파인튜닝 - 불러와서 재학습

        모두연 레스넷 자료

        마지막에 에버리지풀링 넣기
        sparse -> dense 효과
        그래프
        loss같은거 보드에 넣어보기

        miss classfication 된 아이디를 저장해둬볼것
        주로 뭐가 가장 많이 미쓰나는지
        분류를 잘못했는지 데이터가 부족한지 결론낼수잇으.

        비쥬얼라이제이션
        피쳐 중간에 나오는거 어떻게 나오는지 이런거 확인

        1. 다시짜기 - 레이어 세이브 roc test
        2. 아규멘테이션 - 플립 로테 크롭
        3. 비쥬얼라이제이션 & 미스 클래시피케이션 데이터 추출 > 텐서보드 없이
        4. queue
        5. weight 업데이트 디케이, 스텝, 아담+ㅇ스지디

        추가로 되면 레이어추간데 벅참

SyntaxError: invalid syntax (<ipython-input-1-533b2ce24a6d>, line 2)

In [19]:

import numpy as np
import tensorflow as tf

ll = []
print(ll)
ll.append(1)

print(ll)

[]
[1]
